# Movie Recommendation System: Multi-task recommenders

## Install libraries

In [ ]:
!pip install -q tensorflow-recommenders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 3.8 MB/s eta 0:00:00


## Import libraries

In [ ]:
import os
import pprint

from typing import Dict, Text

import pandas as pd
import numpy as np


import tensorflow as tf
import tensorflow_recommenders as tfrs

## Constants

In [ ]:
data_Path = '/content/drive/MyDrive/Colab Notebooks/Recommended_Systems_ Movie/ml-100k'

## Reading Data

In [ ]:
user_info_path = os.path.join(data_Path, 'u.info')

user_info = pd.read_csv(user_info_path, header=None)
user_info.head()

,0
0,943 users
1,1682 items
2,100000 ratings


### Users Data:

In [ ]:
user_data_path = os.path.join(data_Path, 'u.data')

#Reading users file:
column_names = ['user_id', 'item_id', 'rating', 'timestamp']
users_df = pd.read_csv(user_data_path, sep='\t', names=column_names)
# Checking shape of users files and head
print(users_df.shape)
users_df.head()

(100000, 4)


,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


### Movies Data:

In [ ]:
columns = "item_id | title | release date | video release date | "\
             "IMDb URL | unknown | Action | Adventure | Animation | Children's | "\
              "Comedy | Crime | Documentary | Drama | Fantasy | Film-Noir | "\
               "Horror | Musical | Mystery | Romance | Sci-Fi | Thriller | War | Western | "

column_names_2 = columns.split(' | ')
movie_data_path = os.path.join(data_Path, 'u.item')

# Reading the movie data
movie_df = pd.read_csv(movie_data_path, sep='|', header=None, names=column_names_2, encoding='latin-1')
movie_df.drop(movie_df.columns[-1], axis=1, inplace=True)
# Checking shape of movie data and look first 5 rows
print(movie_df.shape)
movie_df.head()

(1682, 24)


,item_id,title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
movie_df = movie_df[['item_id', 'title']]
movie_df.head()

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


### Merger Movie and User data

In [ ]:
# Combining the data on same column
df= pd.merge(users_df, movie_df, on= 'item_id')
df.head()

,user_id,item_id,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [ ]:
df.shape

(100000, 5)

In [ ]:
refined_df = df.groupby(by=['user_id','title'], as_index=False).agg({"rating":"mean"})

refined_df.head()

,user_id,title,rating
0,1,101 Dalmatians (1996),2.0
1,1,12 Angry Men (1957),5.0
2,1,"20,000 Leagues Under the Sea (1954)",3.0
3,1,2001: A Space Odyssey (1968),4.0
4,1,"Abyss, The (1989)",3.0


In [ ]:
refined_df['user_id']=refined_df['user_id'].values.astype(str)
refined_df = tf.data.Dataset.from_tensor_slices(dict(refined_df))

In [ ]:
ratings = refined_df.map(lambda x: {
    "movie_title": x["title"],
    "user_id": x["user_id"],
    "user_rating": x["rating"],
})
movies = refined_df.map(lambda x: x["title"])

In [ ]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

Let's also figure out unique user ids and movie titles present in the data.

In [ ]:
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

unique_movie_titles[:10]

array([b"'Til There Was You (1997)", b'1-900 (1994)',
       b'101 Dalmatians (1996)', b'12 Angry Men (1957)', b'187 (1997)',
       b'2 Days in the Valley (1996)',
       b'20,000 Leagues Under the Sea (1954)',
       b'2001: A Space Odyssey (1968)',
       b'3 Ninjas: High Noon At Mega Mountain (1998)',
       b'39 Steps, The (1935)'], dtype=object)

## A multi-task model

In [ ]:
class MovielensModel(tfrs.models.Model):

  def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
    # We take the loss weights in the constructor: this allows us to instantiate
    # several model objects with different loss weights.

    super().__init__()

    embedding_dimension = 32

    # User and movie models.
    self.movie_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])
    self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # A small model to take in user and movie embeddings and predict ratings.
    self.rating_model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(1),
    ])

    # The tasks.
    self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )
    self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(self.movie_model)
        )
    )

    # The loss weights.
    self.rating_weight = rating_weight
    self.retrieval_weight = retrieval_weight

  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    # pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # pick out the movie features and pass them into the movie model.
    movie_embeddings = self.movie_model(features["movie_title"])

    return (
        user_embeddings,
        movie_embeddings,
        # We apply the multi-layered rating model to a concatentation of
        # user and movie embeddings.
        self.rating_model(
            tf.concat([user_embeddings, movie_embeddings], axis=1)
        ),
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    ratings = features.pop("user_rating")

    user_embeddings, movie_embeddings, rating_predictions = self(features)

    # We compute the loss for each task.
    rating_loss = self.rating_task(
        labels=ratings,
        predictions=rating_predictions,
    )
    retrieval_loss = self.retrieval_task(user_embeddings, movie_embeddings)

    # And combine them using the loss weights.
    return (self.rating_weight * rating_loss
            + self.retrieval_weight * retrieval_loss)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

cached_train = train.shuffle(100_000).batch(8192).cache().prefetch(buffer_size=AUTOTUNE)
cached_test = test.batch(4096).cache().prefetch(buffer_size=AUTOTUNE)

### Rating-specialized model

In [ ]:
model = MovielensModel(rating_weight=1.0, retrieval_weight=0.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [ ]:
model.fit(cached_train, epochs=3)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

### Retrieval-specialized model

In [ ]:
model = MovielensModel(rating_weight=0.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [ ]:
model.fit(cached_train, epochs=3)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

### Joint model

In [ ]:
model = MovielensModel(rating_weight=1.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [ ]:
model.fit(cached_train, epochs=3)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/3
10/10 [==============================] - 551s 54s/step - root_mean_squared_error: 2.2035 - factorized_top_k/top_1_categorical_accuracy: 6.2500e-05 - factorized_top_k/top_5_categorical_accuracy: 7.5000e-05 - factorized_top_k/top_10_categorical_accuracy: 7.5000e-05 - factorized_top_k/top_50_categorical_accuracy: 8.7500e-05 - factorized_top_k/top_100_categorical_accuracy: 1.0000e-04 - loss: 69830.3381 - regularization_loss: 0.0000e+00 - total_loss: 69830.3381
Epoch 2/3
10/10 [==============================] - 542s 54s/step - root_mean_squared_error: 1.2489 - factorized_top_k/top_1_categorical_accuracy: 9.1250e-04 - factorized_top_k/top_5_categorical_accuracy: 9.1250e-04 - factorized_top_k/top_10_categorical_accuracy: 9.1250e-04 - factorized_top_k/top_50_categorical_accuracy: 0.0012 - factorized_top_k/top_100_categorical_accuracy: 0.0022 - loss: 67494.1001 - regularization_loss: 0.0000e+00 - total_loss: 67494.1001
Epoch 3/3
5/5 [==============================] - 140s 27s/step - r

### Making prediction

In [ ]:
trained_movie_embeddings, trained_user_embeddings, predicted_rating = model({
      "user_id": np.array(["90"]),
      "movie_title": np.array(["Dances with Wolves (1990)"])
  })
print(f"Predicted rating:{predicted_rating[0][0]:0.3f}")

Predicted rating:4.333
